In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

In [3]:
from pynq import Overlay
overlay = Overlay('./bitstream/DEC_rLUT28bits.bit')

In [4]:
overlay.ip_dict

{'DEC_rLUT28bits_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xaf78d6f0>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'DEC_rLUT28bits_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S00_AXI',
  'parameters': {'C_S00_AXI_ADDR_WIDTH': '4',
   'C_S00_AXI_BASEADDR': '0x43C00000',
   'C_S00_AXI_DATA_WIDTH': '32',
   'C_S00_AXI_HIGHADDR': '0x43C0FFFF',
   'Component_Name': 'DEC_rLUT28bits_DEC_rLUT28bits_0_0',
   'EDK_IPTYPE': 'PERIPHERAL'},
  'phys_addr': 1136656384,
  'registers': {},
  'state': None,
  'type': 'xilinx.com:user:DEC_rLUT28bits:1.0'}}

In [5]:
from pynq import MMIO
ip = MMIO(0x43C00000, 0x10000)

In [6]:
ip.device

### 將 r_LUT  的location輸出值轉為2補數:

In [7]:
def twos_complement_32bit(x):
    """將 32-bit 補數整數轉為有號值"""
    if x >= (1 << 31):
        return x - (1 << 32)
    else:
        return x

#### Test:

In [8]:
remainder_1 = 14443

ip.write_mm(0, remainder_1)

location_1 = ip.read_mm(4)
location_2 = ip.read_mm(8)


print(twos_complement_32bit(location_1))
print(twos_complement_32bit(location_2))

24
-35


### 開始則中查表驗證

## 折衷查表:
### 用 Single Error Correction 解 Double Error Correction (A 取Double_AWE的 A)

#### 1.取 A的bits $\approx$ N的bits $\Rightarrow$ A = 18613  (30bits), N的範圍: 0 ~ 1073741823 (30bits)

### Define function
* (Not including 2's complement range

In [9]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    return number.bit_length()


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value

## 把重複的 AWE 去除
def remove_duplicates(input_list):
    unique_list = []
    for number in input_list:
        if number not in unique_list:
            unique_list.append(number)
    return unique_list


def get_sign(number):
    return 1 if number > 0 else -1 if number < 0 else 0


print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


In [10]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data1 = (1 << 28) - 21    ## 
data2 = (1 << 28) - 19
data3 = (1 << 28) - 15
AN = 17619
input1 = data1*AN
input2 = data2*AN
input3 = data3*AN
bits1 = bits_needed(input1)
bits2 = bits_needed(input2)
bits3 = bits_needed(input3)
print("inputs1的位元數:", bits1)
print("inputs2的位元數:", bits2)
print("inputs3的位元數:", bits3)

inputs1的位元數: 43
inputs2的位元數: 43
inputs3的位元數: 43


### 產生1-bit AWE (Arithmetic-Weight Error)
* 並產生 Delta Q 的 list

In [11]:
AWE1_1=[]
e1_1=0     ## 錯誤數量
e1_data1=[]
h = [];   #位置
for i in range(0, bits1):
    e1_data1.insert(e1_1 , input1 + 2**i)
    AWE1_1.insert(e1_1, 2**i)
    e1_1 = e1_1 + 1
    e1_data1.insert(e1_1 , input1 - 2**i)
    AWE1_1.insert(e1_1, -2**i)
    e1_1 = e1_1 + 1
    h.insert(i,i+1)

AWE1_2=[]
e1_2=0     ## 錯誤數量
e1_data2=[]
for i in range(0, bits2):
    e1_data2.insert(e1_2 , input2 + 2**i)
    AWE1_2.insert(e1_2, 2**i)
    e1_2 = e1_2 + 1
    e1_data2.insert(e1_2 , input2 - 2**i)
    AWE1_2.insert(e1_2, -2**i)
    e1_2 = e1_2 + 1
    h.insert(i,i+1)
    
AWE1_3=[]
e1_3=0     ## 錯誤數量
e1_data3=[]
for i in range(0, bits3):
    e1_data3.insert(e1_3 , input3 + 2**i)
    AWE1_3.insert(e1_3, 2**i)
    e1_3 = e1_3 + 1
    e1_data3.insert(e1_3 , input3 - 2**i)
    AWE1_3.insert(e1_3, -2**i)
    e1_3 = e1_3 + 1
    h.insert(i,i+1)

h_awe = [val for num in h for val in (num, -num)]
print("產生1-bit AWE:", AWE1_1)
print("不同輸入資料的AWE是否相等:",AWE1_1 == AWE1_2 == AWE1_3)
print(h_awe)
print(np.size(AWE1_1))

產生1-bit AWE: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 8589934592, -8589934592, 17179869184, -17179869184, 34359738368, -34359738368, 68719476736, -68719476736, 137438953472, -137438953472, 274877906944, -274877906944, 549755813888, -549755813888, 1099511627776, -1099511627776, 2199023255552, -2199023255552, 4398046511104, -4398046511104]
不同輸入資料的AWE是否相等: True
[1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 15, -15, 16, -16, 17, -17, 18, -18, 19, -19, 20, -2

### 產生SEC的模數:

In [12]:
SEC_R = []
AWE1_R= []
for m in range(0,e1_1):
    mod = e1_data1[m] % AN
    SEC_R.append(mod)

for n in range(0,e1_1):
    mod_awe = AWE1_3[n] % AN
    AWE1_R.append(mod_awe)


print("SEC的模數:\n",SEC_R)
print("AWE1的模數:\n",SEC_R)
print("AWE1的模數是否等於SEC模數:", SEC_R == AWE1_R)
print("SEC的大小:",np.size(SEC_R))

SEC的模數:
 [1, 17618, 2, 17617, 4, 17615, 8, 17611, 16, 17603, 32, 17587, 64, 17555, 128, 17491, 256, 17363, 512, 17107, 1024, 16595, 2048, 15571, 4096, 13523, 8192, 9427, 16384, 1235, 15149, 2470, 12679, 4940, 7739, 9880, 15478, 2141, 13337, 4282, 9055, 8564, 491, 17128, 982, 16637, 1964, 15655, 3928, 13691, 7856, 9763, 15712, 1907, 13805, 3814, 9991, 7628, 2363, 15256, 4726, 12893, 9452, 8167, 1285, 16334, 2570, 15049, 5140, 12479, 10280, 7339, 2941, 14678, 5882, 11737, 11764, 5855, 5909, 11710, 11818, 5801, 6017, 11602, 12034, 5585]
AWE1的模數:
 [1, 17618, 2, 17617, 4, 17615, 8, 17611, 16, 17603, 32, 17587, 64, 17555, 128, 17491, 256, 17363, 512, 17107, 1024, 16595, 2048, 15571, 4096, 13523, 8192, 9427, 16384, 1235, 15149, 2470, 12679, 4940, 7739, 9880, 15478, 2141, 13337, 4282, 9055, 8564, 491, 17128, 982, 16637, 1964, 15655, 3928, 13691, 7856, 9763, 15712, 1907, 13805, 3814, 9991, 7628, 2363, 15256, 4726, 12893, 9452, 8167, 1285, 16334, 2570, 15049, 5140, 12479, 10280, 7339, 2941, 1467

### 產生2-bit AWE (Arithmetic-Weight Error)

In [13]:
AWE_2=[]
e2=0     ## 錯誤數量
p=0      ## 錯誤位置
e2_data=[]
for i in range(0, bits1):
    for j in range(i+1, bits1):
        e2_data.insert(e2 , (input1 + 2**i + 2**j))
        AWE_2.insert(e2 , (2**i + 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 + 2**i - 2**j))
        AWE_2.insert(e2 , (2**i - 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i + 2**j))
        AWE_2.insert(e2 , (-2**i + 2**j))                
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i -2**j))
        AWE_2.insert(e2 , (-2**i - 2**j))
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        p = p + 1


print(e2)
print(p)
print("產生2-bits AWE:",AWE_2)
print("產生2-bits 錯誤資料:",e2_data)

第 0 筆錯誤資料: 4729563929268
第 0 筆算術錯誤: 3
第 1 筆錯誤資料: 4729563929264
第 1 筆算術錯誤: -1
第 2 筆錯誤資料: 4729563929266
第 2 筆算術錯誤: 1
第 3 筆錯誤資料: 4729563929262
第 3 筆算術錯誤: -3
第 4 筆錯誤資料: 4729563929270
第 4 筆算術錯誤: 5
第 5 筆錯誤資料: 4729563929262
第 5 筆算術錯誤: -3
第 6 筆錯誤資料: 4729563929268
第 6 筆算術錯誤: 3
第 7 筆錯誤資料: 4729563929260
第 7 筆算術錯誤: -5
第 8 筆錯誤資料: 4729563929274
第 8 筆算術錯誤: 9
第 9 筆錯誤資料: 4729563929258
第 9 筆算術錯誤: -7
第 10 筆錯誤資料: 4729563929272
第 10 筆算術錯誤: 7
第 11 筆錯誤資料: 4729563929256
第 11 筆算術錯誤: -9
第 12 筆錯誤資料: 4729563929282
第 12 筆算術錯誤: 17
第 13 筆錯誤資料: 4729563929250
第 13 筆算術錯誤: -15
第 14 筆錯誤資料: 4729563929280
第 14 筆算術錯誤: 15
第 15 筆錯誤資料: 4729563929248
第 15 筆算術錯誤: -17
第 16 筆錯誤資料: 4729563929298
第 16 筆算術錯誤: 33
第 17 筆錯誤資料: 4729563929234
第 17 筆算術錯誤: -31
第 18 筆錯誤資料: 4729563929296
第 18 筆算術錯誤: 31
第 19 筆錯誤資料: 4729563929232
第 19 筆算術錯誤: -33
第 20 筆錯誤資料: 4729563929330
第 20 筆算術錯誤: 65
第 21 筆錯誤資料: 4729563929202
第 21 筆算術錯誤: -63
第 22 筆錯誤資料: 4729563929328
第 22 筆算術錯誤: 63
第 23 筆錯誤資料: 4729563929200
第 23 筆算術錯誤: -65
第 24 筆錯誤資料: 4729563929394
第 24 筆算術錯誤

第 358 筆算術錯誤: 508
第 359 筆錯誤資料: 4729563928749
第 359 筆算術錯誤: -516
第 360 筆錯誤資料: 4729563930293
第 360 筆算術錯誤: 1028
第 361 筆錯誤資料: 4729563928245
第 361 筆算術錯誤: -1020
第 362 筆錯誤資料: 4729563930285
第 362 筆算術錯誤: 1020
第 363 筆錯誤資料: 4729563928237
第 363 筆算術錯誤: -1028
第 364 筆錯誤資料: 4729563931317
第 364 筆算術錯誤: 2052
第 365 筆錯誤資料: 4729563927221
第 365 筆算術錯誤: -2044
第 366 筆錯誤資料: 4729563931309
第 366 筆算術錯誤: 2044
第 367 筆錯誤資料: 4729563927213
第 367 筆算術錯誤: -2052
第 368 筆錯誤資料: 4729563933365
第 368 筆算術錯誤: 4100
第 369 筆錯誤資料: 4729563925173
第 369 筆算術錯誤: -4092
第 370 筆錯誤資料: 4729563933357
第 370 筆算術錯誤: 4092
第 371 筆錯誤資料: 4729563925165
第 371 筆算術錯誤: -4100
第 372 筆錯誤資料: 4729563937461
第 372 筆算術錯誤: 8196
第 373 筆錯誤資料: 4729563921077
第 373 筆算術錯誤: -8188
第 374 筆錯誤資料: 4729563937453
第 374 筆算術錯誤: 8188
第 375 筆錯誤資料: 4729563921069
第 375 筆算術錯誤: -8196
第 376 筆錯誤資料: 4729563945653
第 376 筆算術錯誤: 16388
第 377 筆錯誤資料: 4729563912885
第 377 筆算術錯誤: -16380
第 378 筆錯誤資料: 4729563945645
第 378 筆算術錯誤: 16380
第 379 筆錯誤資料: 4729563912877
第 379 筆算術錯誤: -16388
第 380 筆錯誤資料: 47295639620

第 690 筆錯誤資料: 4729563962017
第 690 筆算術錯誤: 32752
第 691 筆錯誤資料: 4729563896481
第 691 筆算術錯誤: -32784
第 692 筆錯誤資料: 4729563994817
第 692 筆算術錯誤: 65552
第 693 筆錯誤資料: 4729563863745
第 693 筆算術錯誤: -65520
第 694 筆錯誤資料: 4729563994785
第 694 筆算術錯誤: 65520
第 695 筆錯誤資料: 4729563863713
第 695 筆算術錯誤: -65552
第 696 筆錯誤資料: 4729564060353
第 696 筆算術錯誤: 131088
第 697 筆錯誤資料: 4729563798209
第 697 筆算術錯誤: -131056
第 698 筆錯誤資料: 4729564060321
第 698 筆算術錯誤: 131056
第 699 筆錯誤資料: 4729563798177
第 699 筆算術錯誤: -131088
第 700 筆錯誤資料: 4729564191425
第 700 筆算術錯誤: 262160
第 701 筆錯誤資料: 4729563667137
第 701 筆算術錯誤: -262128
第 702 筆錯誤資料: 4729564191393
第 702 筆算術錯誤: 262128
第 703 筆錯誤資料: 4729563667105
第 703 筆算術錯誤: -262160
第 704 筆錯誤資料: 4729564453569
第 704 筆算術錯誤: 524304
第 705 筆錯誤資料: 4729563404993
第 705 筆算術錯誤: -524272
第 706 筆錯誤資料: 4729564453537
第 706 筆算術錯誤: 524272
第 707 筆錯誤資料: 4729563404961
第 707 筆算術錯誤: -524304
第 708 筆錯誤資料: 4729564977857
第 708 筆算術錯誤: 1048592
第 709 筆錯誤資料: 4729562880705
第 709 筆算術錯誤: -1048560
第 710 筆錯誤資料: 4729564977825
第 710 筆算術錯誤: 1048560
第 711 

第 939 筆算術錯誤: -1099511627808
第 940 筆錯誤資料: 6928587184849
第 940 筆算術錯誤: 2199023255584
第 941 筆錯誤資料: 2530540673745
第 941 筆算術錯誤: -2199023255520
第 942 筆錯誤資料: 6928587184785
第 942 筆算術錯誤: 2199023255520
第 943 筆錯誤資料: 2530540673681
第 943 筆算術錯誤: -2199023255584
第 944 筆錯誤資料: 9127610440401
第 944 筆算術錯誤: 4398046511136
第 945 筆錯誤資料: 331517418193
第 945 筆算術錯誤: -4398046511072
第 946 筆錯誤資料: 9127610440337
第 946 筆算術錯誤: 4398046511072
第 947 筆錯誤資料: 331517418129
第 947 筆算術錯誤: -4398046511136
第 948 筆錯誤資料: 4729563929457
第 948 筆算術錯誤: 192
第 949 筆錯誤資料: 4729563929201
第 949 筆算術錯誤: -64
第 950 筆錯誤資料: 4729563929329
第 950 筆算術錯誤: 64
第 951 筆錯誤資料: 4729563929073
第 951 筆算術錯誤: -192
第 952 筆錯誤資料: 4729563929585
第 952 筆算術錯誤: 320
第 953 筆錯誤資料: 4729563929073
第 953 筆算術錯誤: -192
第 954 筆錯誤資料: 4729563929457
第 954 筆算術錯誤: 192
第 955 筆錯誤資料: 4729563928945
第 955 筆算術錯誤: -320
第 956 筆錯誤資料: 4729563929841
第 956 筆算術錯誤: 576
第 957 筆錯誤資料: 4729563928817
第 957 筆算術錯誤: -448
第 958 筆錯誤資料: 4729563929713
第 958 筆算術錯誤: 448
第 959 筆錯誤資料: 4729563928689
第 959 筆算術錯誤: -576
第 960 

第 1189 筆算術錯誤: -4294967168
第 1190 筆錯誤資料: 4733858896433
第 1190 筆算術錯誤: 4294967168
第 1191 筆錯誤資料: 4725268961841
第 1191 筆算術錯誤: -4294967424
第 1192 筆錯誤資料: 4738153863985
第 1192 筆算術錯誤: 8589934720
第 1193 筆錯誤資料: 4720973994801
第 1193 筆算術錯誤: -8589934464
第 1194 筆錯誤資料: 4738153863729
第 1194 筆算術錯誤: 8589934464
第 1195 筆錯誤資料: 4720973994545
第 1195 筆算術錯誤: -8589934720
第 1196 筆錯誤資料: 4746743798577
第 1196 筆算術錯誤: 17179869312
第 1197 筆錯誤資料: 4712384060209
第 1197 筆算術錯誤: -17179869056
第 1198 筆錯誤資料: 4746743798321
第 1198 筆算術錯誤: 17179869056
第 1199 筆錯誤資料: 4712384059953
第 1199 筆算術錯誤: -17179869312
第 1200 筆錯誤資料: 4763923667761
第 1200 筆算術錯誤: 34359738496
第 1201 筆錯誤資料: 4695204191025
第 1201 筆算術錯誤: -34359738240
第 1202 筆錯誤資料: 4763923667505
第 1202 筆算術錯誤: 34359738240
第 1203 筆錯誤資料: 4695204190769
第 1203 筆算術錯誤: -34359738496
第 1204 筆錯誤資料: 4798283406129
第 1204 筆算術錯誤: 68719476864
第 1205 筆錯誤資料: 4660844452657
第 1205 筆算術錯誤: -68719476608
第 1206 筆錯誤資料: 4798283405873
第 1206 筆算術錯誤: 68719476608
第 1207 筆錯誤資料: 4660844452401
第 1207 筆算術錯誤: -68719476864

...

第 2438 筆錯誤資料: 4729597221553
第 2438 筆算術錯誤: 33292288
第 2439 筆錯誤資料: 4729530112689
第 2439 筆算術錯誤: -33816576
第 2440 筆錯誤資料: 4729631300273
第 2440 筆算術錯誤: 67371008
第 2441 筆錯誤資料: 4729497082545
第 2441 筆算術錯誤: -66846720
第 2442 筆錯誤資料: 4729630775985
第 2442 筆算術錯誤: 66846720
第 2443 筆錯誤資料: 4729496558257
第 2443 筆算術錯誤: -67371008
第 2444 筆錯誤資料: 4729698409137
第 2444 筆算術錯誤: 134479872
第 2445 筆錯誤資料: 4729429973681
第 2445 筆算術錯誤: -133955584
第 2446 筆錯誤資料: 4729697884849
第 2446 筆算術錯誤: 133955584
第 2447 筆錯誤資料: 4729429449393
第 2447 筆算術錯誤: -134479872
第 2448 筆錯誤資料: 4729832626865
第 2448 筆算術錯誤: 268697600
第 2449 筆錯誤資料: 4729295755953
第 2449 筆算術錯誤: -268173312
第 2450 筆錯誤資料: 4729832102577
第 2450 筆算術錯誤: 268173312
第 2451 筆錯誤資料: 4729295231665
第 2451 筆算術錯誤: -268697600
第 2452 筆錯誤資料: 4730101062321
第 2452 筆算術錯誤: 537133056
第 2453 筆錯誤資料: 4729027320497
第 2453 筆算術錯誤: -536608768
第 2454 筆錯誤資料: 4730100538033
第 2454 筆算術錯誤: 536608768
第 2455 筆錯誤資料: 4729026796209
第 2455 筆算術錯誤: -537133056
第 2456 筆錯誤資料: 4730637933233
第 2456 筆算術錯誤: 1074003968
第 2457 筆

第 2688 筆錯誤資料: 4729570220721
第 2688 筆算術錯誤: 6291456
第 2689 筆錯誤資料: 4729561832113
第 2689 筆算術錯誤: -2097152
第 2690 筆錯誤資料: 4729566026417
第 2690 筆算術錯誤: 2097152
第 2691 筆錯誤資料: 4729557637809
第 2691 筆算術錯誤: -6291456
第 2692 筆錯誤資料: 4729574415025
第 2692 筆算術錯誤: 10485760
第 2693 筆錯誤資料: 4729557637809
第 2693 筆算術錯誤: -6291456
第 2694 筆錯誤資料: 4729570220721
第 2694 筆算術錯誤: 6291456
第 2695 筆錯誤資料: 4729553443505
第 2695 筆算術錯誤: -10485760
第 2696 筆錯誤資料: 4729582803633
第 2696 筆算術錯誤: 18874368
第 2697 筆錯誤資料: 4729549249201
第 2697 筆算術錯誤: -14680064
第 2698 筆錯誤資料: 4729578609329
第 2698 筆算術錯誤: 14680064
第 2699 筆錯誤資料: 4729545054897
第 2699 筆算術錯誤: -18874368
第 2700 筆錯誤資料: 4729599580849
第 2700 筆算術錯誤: 35651584
第 2701 筆錯誤資料: 4729532471985
第 2701 筆算術錯誤: -31457280
第 2702 筆錯誤資料: 4729595386545
第 2702 筆算術錯誤: 31457280
第 2703 筆錯誤資料: 4729528277681
第 2703 筆算術錯誤: -35651584
第 2704 筆錯誤資料: 4729633135281
第 2704 筆算術錯誤: 69206016
第 2705 筆錯誤資料: 4729498917553
第 2705 筆算術錯誤: -65011712
第 2706 筆錯誤資料: 4729628940977
第 2706 筆算術錯誤: 65011712
第 2707 筆錯誤資料: 4729494723249


第 2937 筆算術錯誤: -117440512
第 2938 筆錯誤資料: 4729681369777
第 2938 筆算術錯誤: 117440512
第 2939 筆錯誤資料: 4729412934321
第 2939 筆算術錯誤: -150994944
第 2940 筆錯誤資料: 4729849141937
第 2940 筆算術錯誤: 285212672
第 2941 筆錯誤資料: 4729312271025
第 2941 筆算術錯誤: -251658240
第 2942 筆錯誤資料: 4729815587505
第 2942 筆算術錯誤: 251658240
第 2943 筆錯誤資料: 4729278716593
第 2943 筆算術錯誤: -285212672
第 2944 筆錯誤資料: 4730117577393
第 2944 筆算術錯誤: 553648128
第 2945 筆錯誤資料: 4729043835569
第 2945 筆算術錯誤: -520093696
第 2946 筆錯誤資料: 4730084022961
第 2946 筆算術錯誤: 520093696
第 2947 筆錯誤資料: 4729010281137
第 2947 筆算術錯誤: -553648128
第 2948 筆錯誤資料: 4730654448305
第 2948 筆算術錯誤: 1090519040
第 2949 筆錯誤資料: 4728506964657
第 2949 筆算術錯誤: -1056964608
第 2950 筆錯誤資料: 4730620893873
第 2950 筆算術錯誤: 1056964608
第 2951 筆錯誤資料: 4728473410225
第 2951 筆算術錯誤: -1090519040
第 2952 筆錯誤資料: 4731728190129
第 2952 筆算術錯誤: 2164260864
第 2953 筆錯誤資料: 4727433222833
第 2953 筆算術錯誤: -2130706432
第 2954 筆錯誤資料: 4731694635697
第 2954 筆算術錯誤: 2130706432
第 2955 筆錯誤資料: 4727399668401
第 2955 筆算術錯誤: -2164260864
第 2956 筆錯誤資料: 47338756

第 3187 筆算術錯誤: -2199157473280
第 3188 筆錯誤資料: 9127744658097
第 3188 筆算術錯誤: 4398180728832
第 3189 筆錯誤資料: 331651635889
第 3189 筆算術錯誤: -4397912293376
第 3190 筆錯誤資料: 9127476222641
第 3190 筆算術錯誤: 4397912293376
第 3191 筆錯誤資料: 331383200433
第 3191 筆算術錯誤: -4398180728832
第 3192 筆錯誤資料: 4730369235633
第 3192 筆算術錯誤: 805306368
第 3193 筆錯誤資料: 4729295493809
第 3193 筆算術錯誤: -268435456
第 3194 筆錯誤資料: 4729832364721
第 3194 筆算術錯誤: 268435456
第 3195 筆錯誤資料: 4728758622897
第 3195 筆算術錯誤: -805306368
第 3196 筆錯誤資料: 4730906106545
第 3196 筆算術錯誤: 1342177280
第 3197 筆錯誤資料: 4728758622897
第 3197 筆算術錯誤: -805306368
第 3198 筆錯誤資料: 4730369235633
第 3198 筆算術錯誤: 805306368
第 3199 筆錯誤資料: 4728221751985
第 3199 筆算術錯誤: -1342177280
第 3200 筆錯誤資料: 4731979848369
第 3200 筆算術錯誤: 2415919104
第 3201 筆錯誤資料: 4727684881073
第 3201 筆算術錯誤: -1879048192
第 3202 筆錯誤資料: 4731442977457
第 3202 筆算術錯誤: 1879048192
第 3203 筆錯誤資料: 4727148010161
第 3203 筆算術錯誤: -2415919104
第 3204 筆錯誤資料: 4734127332017
第 3204 筆算術錯誤: 4563402752
第 3205 筆錯誤資料: 4725537397425
第 3205 筆算術錯誤: -4026531840
第 32

第 3437 筆錯誤資料: 4703794125489
第 3437 筆算術錯誤: -25769803776
第 3438 筆錯誤資料: 4755333733041
第 3438 筆算術錯誤: 25769803776
第 3439 筆錯誤資料: 4686614256305
第 3439 筆算術錯誤: -42949672960
第 3440 筆錯誤資料: 4806873340593
第 3440 筆算術錯誤: 77309411328
第 3441 筆錯誤資料: 4669434387121
第 3441 筆算術錯誤: -60129542144
第 3442 筆錯誤資料: 4789693471409
第 3442 筆算術錯誤: 60129542144
第 3443 筆錯誤資料: 4652254517937
第 3443 筆算術錯誤: -77309411328
第 3444 筆錯誤資料: 4875592817329
第 3444 筆算術錯誤: 146028888064
第 3445 筆錯誤資料: 4600714910385
第 3445 筆算術錯誤: -128849018880
第 3446 筆錯誤資料: 4858412948145
第 3446 筆算術錯誤: 128849018880
第 3447 筆錯誤資料: 4583535041201
第 3447 筆算術錯誤: -146028888064
第 3448 筆錯誤資料: 5013031770801
第 3448 筆算術錯誤: 283467841536
第 3449 筆錯誤資料: 4463275956913
第 3449 筆算術錯誤: -266287972352
第 3450 筆錯誤資料: 4995851901617
第 3450 筆算術錯誤: 266287972352
第 3451 筆錯誤資料: 4446096087729
第 3451 筆算術錯誤: -283467841536
第 3452 筆錯誤資料: 5287909677745
第 3452 筆算術錯誤: 558345748480
第 3453 筆錯誤資料: 4188398049969
第 3453 筆算術錯誤: -541165879296
第 3454 筆錯誤資料: 5270729808561
第 3454 筆算術錯誤: 541165879296
第 3455 筆

In [14]:
DEC_R = []
for m in range(0,e2):
    mod = e2_data[m] % AN
    DEC_R.append(mod)

print("DEC_R的模數:\n",DEC_R)
print(np.size(DEC_R))

DEC_R的模數:
 [3, 17618, 1, 17616, 5, 17616, 3, 17614, 9, 17612, 7, 17610, 17, 17604, 15, 17602, 33, 17588, 31, 17586, 65, 17556, 63, 17554, 129, 17492, 127, 17490, 257, 17364, 255, 17362, 513, 17108, 511, 17106, 1025, 16596, 1023, 16594, 2049, 15572, 2047, 15570, 4097, 13524, 4095, 13522, 8193, 9428, 8191, 9426, 16385, 1236, 16383, 1234, 15150, 2471, 15148, 2469, 12680, 4941, 12678, 4939, 7740, 9881, 7738, 9879, 15479, 2142, 15477, 2140, 13338, 4283, 13336, 4281, 9056, 8565, 9054, 8563, 492, 17129, 490, 17127, 983, 16638, 981, 16636, 1965, 15656, 1963, 15654, 3929, 13692, 3927, 13690, 7857, 9764, 7855, 9762, 15713, 1908, 15711, 1906, 13806, 3815, 13804, 3813, 9992, 7629, 9990, 7627, 2364, 15257, 2362, 15255, 4727, 12894, 4725, 12892, 9453, 8168, 9451, 8166, 1286, 16335, 1284, 16333, 2571, 15050, 2569, 15048, 5141, 12480, 5139, 12478, 10281, 7340, 10279, 7338, 2942, 14679, 2940, 14677, 5883, 11738, 5881, 11736, 11765, 5856, 11763, 5854, 5910, 11711, 5908, 11709, 11819, 5802, 11817, 5800, 

In [15]:
total_AWE = AWE1_1 + AWE_2
e_total = e1_data1 + e2_data
R_total = SEC_R + DEC_R
print("還未去除重複error data的數量:",np.size(e_total))
print("還未去除重複餘數的數量:", np.size(R_total))

## 把重複的 AWE 及 error_data 去除
AWE = remove_duplicates(total_AWE)
new_e_total = remove_duplicates(e_total)
new_R_total = remove_duplicates(R_total)
error_total_count = np.size(new_e_total)
print("已去除重複error data的數量:",error_total_count)
print("已未去除重複餘數的數量:",np.size(new_R_total))
print("已未去除重複AWE的數量:",np.size(AWE))
print("所有的AWE為:\n",AWE)
print("所有算術錯誤的資料:\n", new_e_total)

還未去除重複error data的數量: 3698
還未去除重複餘數的數量: 3698
已去除重複error data的數量: 3532
已未去除重複餘數的數量: 3532
已未去除重複AWE的數量: 3532
所有的AWE為:
 [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 8589934592, -8589934592, 17179869184, -17179869184, 34359738368, -34359738368, 68719476736, -68719476736, 137438953472, -137438953472, 274877906944, -274877906944, 549755813888, -549755813888, 1099511627776, -1099511627776, 2199023255552, -2199023255552, 4398046511104, -4398046511104, 3, -3, 5, -5, 9, -7, 7, -9, 17, -15, 15, -17, 33, -31, 31, -33, 65, -63, 63, -

### 所有AWE數量: $4*C^{bits}_{2}$ $-$ $2*(bits-1)$ - $2*(bits-2)$
* $2*(bits-1)$: 所有的 single AWE都可以分解成已重複的 double AWE, 除了MSB例外
* EX: $(+,0)$ 只能分解成 $(0,++)$ , $(-,0)$ 只能分解成 $(0,--)$
* $2*(bits-2)$: double AWE重複的個數
* EX: $(+,0,-)$ = $(0,+,+)$ and $(-,0,+)$ = $(0,-,-)$

### Trade-off Algorithm: 2025/05/27 使用 Vivado HLX IP 驗證
* 此解出來的數字為位置及正負號 EX: $-1$ 代表 $-2^0$ , $-5$ 代表 $-2^4$ 以此類推

In [16]:
import math
import time
### 給定一個DEC_R, 找錯誤位置h1及h2(包含錯誤正負號)
import numpy as np

h1 = 0; h2 = 0; h_temp = 0
possible_h1 = []; possible_h2 = []; possible_AWE = []
output = []; possible_SE_h1=[]

start_time = time.time()  # 記錄開始時間
for R, error in zip(new_R_total, new_e_total): 
    ip.write_mm(0, R)
    
    h1 = twos_complement_32bit(ip.read_mm(4))                  
    h2 = twos_complement_32bit(ip.read_mm(8))
    
    s2 = get_sign(h2)
    s1 = get_sign(h1)
    
    possible_h1.append(h1)                  
    possible_h2.append(h2)                  
    output.append((error - (s1 * 2**(abs(h1)-1)) - (s2 * 2**(abs(h2)-1))) // AN)

            
end_time = time.time()  # 記錄結束時間
elapsed_time = end_time - start_time  # 計算執行時間
print(f"執行時間: {elapsed_time:.6f} 秒")
print("更正後的值是否和原data相同:", data1 == output.pop())
print(elapsed_time / np.size(possible_h1))
## print(output)
### print(possible_h1).
### print(possible_h2)
##  print("更正後的值:",output)
print(np.size(possible_h1))
print(np.size(possible_h2))
##print("輸入的 error data的總數是否和經過演算法後相同:", error_total_count == np.size(possible_h1))
##print(possible_SE_h1)

執行時間: 0.384550 秒
更正後的值是否和原data相同: True
0.00010887608522737013
3532
3532
